## Direct API calls to Gemini

In [1]:
# import base64
import os
from google import genai
# from google.genai import types


def generate(prompt):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )

    print(response.text)
    

generate("What is 1 + 3?")


1 + 3 = 4



In [2]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)


Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [3]:
generate(prompt)

Oh dear, I'm quite frustrated because my blender lid came off and splashed smoothie all over my kitchen walls! To add to that, the warranty unfortunately doesn't cover the cost of cleaning up my kitchen. I could really use some assistance right now, if you don't mind.



## Chat API : LangChain

In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]

gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [2]:
from langchain_core.prompts import PromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# Instantiation using from_template (recommended)
prompt_template = PromptTemplate.from_template(template_string)

print(prompt_template.template)
print(prompt_template.input_variables)
print(prompt_template)

Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```

['style', 'text']
input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


In [3]:
customer_style = """Super Polite English Gentleman"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_prompt(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(customer_messages)

<class 'langchain_core.prompt_values.StringPromptValue'>
text="Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [4]:
customer_response = gemini.invoke(customer_messages)
print(customer_response)

content='Good heavens, I must confess I am in a rather unfortunate predicament. It appears that, during the preparation of a smoothie, the lid of my blender became rather... detached, resulting in a most unwelcome spattering of the aforementioned beverage upon the kitchen walls. A truly distressing situation, I assure you.\n\nTo further compound matters, it seems the warranty, alas, does not extend to the rather considerable expense of restoring my kitchen to its former, pristine condition. Therefore, I find myself in the rather awkward position of requiring some assistance. I would be most grateful for any guidance you might be able to offer at your earliest convenience. Thank you ever so much.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-b816067c-8d37-41da-a2f4-6b2472323b74-0' usage_metadata={'input_tokens': 79, 'output_tokens': 134, 'total_tokens': 213, 'input_token_detai

In [8]:
customer_response = gemini.invoke("Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")

In [9]:
print(customer_response)

content='Good heavens, I must confess I am in a rather unfortunate predicament. It appears that, during the preparation of a smoothie, the lid of my blender became rather... detached, resulting in a most unwelcome spattering of the aforementioned beverage upon the kitchen walls. A truly distressing situation, I assure you.\n\nTo further compound matters, it seems the warranty, alas, does not extend to the rather considerable expense of restoring my kitchen to its former, pristine condition. Therefore, I find myself in the rather awkward position of requiring some assistance. I would be most grateful for any guidance you might be able to offer at your earliest convenience. Thank you ever so much.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-85a04cae-ed92-41a8-95b2-e0645d1bf46d-0' usage_metadata={'input_tokens': 79, 'output_tokens': 134, 'total_tokens': 213, 'input_token_detai

In [10]:
messages = [
    ("system", "Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman."),
    ("human", "```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```."),
]
gemini.invoke(messages).content


"Good heavens, I say! I find myself in a rather unfortunate predicament, you see. During the preparation of a smoothie, the lid of my blender, in a most unexpected fashion, detached itself, resulting in a rather messy splattering of the kitchen walls. A most unwelcome occurrence, indeed!\n\nTo add to my distress, I've discovered that the warranty, alas, does not extend to the cost of rectifying this rather sticky situation. Therefore, I find myself in need of your kind assistance at this very moment. I would be most grateful for any guidance you might offer."

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_template_string = [
    ("system", "Translate the text that is delimited by triple backticks into a style that is {style}."),
    ("human", "```{text}```"),
]
chat_prompt_template = ChatPromptTemplate(chat_template_string)
customer_messages = chat_prompt_template.invoke(
                        {
                            "style": customer_style,
                            "text": customer_email,
                        }
                    )
print(customer_messages.messages)
gemini.invoke(customer_messages).content

[SystemMessage(content='Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman.', additional_kwargs={}, response_metadata={}), HumanMessage(content="```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```", additional_kwargs={}, response_metadata={})]


"Good heavens, I say! I find myself in a rather unfortunate predicament, you see. During the preparation of a smoothie, the lid of my blender, in a most unexpected fashion, detached itself and, alas, splattered the walls of my kitchen with the aforementioned concoction. A most unwelcome event, indeed!\n\nTo add insult to injury, I'm afraid the warranty, in its current form, does not extend to the cost of rectifying this rather messy situation. Therefore, I find myself in need of your esteemed assistance at this very moment. I would be most grateful for any guidance you might be able to offer."

In [13]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_prompt(
    style=service_style_pirate,
    text=service_reply)

print(service_messages)

text="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"


In [14]:
service_response = gemini.invoke(service_messages)
print(service_response)

content="Ahoy there, matey!\n\nI be havin' to inform ye that, alas, the warranty be not coverin' the cleanin' expenses for yer galley. It seems a bit o' a blunder occurred with yer blender, and 'tis yer own doin' for forgettin' to secure the lid afore givin' 'er a whirl. A bit o' bad luck, I'm afraid! Fair winds to ye!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-c94d0d8f-ed79-42bd-9470-7c25a8014f6f-0' usage_metadata={'input_tokens': 73, 'output_tokens': 91, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}}


In [15]:
service_messages = chat_prompt_template.invoke(
                        {
                            "style": service_style_pirate,
                            "text": service_reply,
                        }
                    )
service_response = gemini.invoke(service_messages)
print(service_response)

content="Ahoy there, matey!\n\nRegarding yer kitchen, I be afraid the warranty doesn't cover the cleaning expenses. It seems the mishap with yer blender, lackin' a lid afore startin' it, be the cause o' the mess. A bit o' misfortune on yer part, I'd say. Fair winds to ye!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-fa750c18-240f-4915-9843-bb95514534c0-0' usage_metadata={'input_tokens': 70, 'output_tokens': 72, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}}
